In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
# pylint: disable=missing-docstring
import sys
sys.path.append('../../')
import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test as input_test
import math
import numpy as np
from i3d import InceptionI3d
from utils import *

# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1, 'Batch size.')
flags.DEFINE_integer('num_frame_per_clib', 64, 'Nummber of frames per clib')
flags.DEFINE_integer('crop_size', 224, 'Crop_size')
flags.DEFINE_integer('rgb_channels', 3, 'Channels for input')
flags.DEFINE_integer('classics', 51, 'The num of class')
FLAGS = flags.FLAGS

nn = 64
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
outfile = './features_trainval/hmdb_flow_64/'
if not os.path.exists(outfile):
    os.makedirs(outfile)
    
def run_training():
    # Get the sets of images and labels for training, validation, and
    # Tell TensorFlow that the model will be built into the default Graph.
    pre_model_save_dir = "./best_models/flow_imagenet_40000_6_64_0.0001_decay2"
    test_list_file = '/media/ee401_2/SPML4/temporal_action/I3D-Tensorflow/list/hmdb_list/testlist1.list'
    file = list(open(test_list_file, 'r'))
    num_test_videos = len(file)
    print("Number of test videos={}".format(num_test_videos))
    with tf.Graph().as_default():
        rgb_images_placeholder, flow_images_placeholder, labels_placeholder, is_training = placeholder_inputs(
                        FLAGS.batch_size * gpu_num,
                        FLAGS.num_frame_per_clib,
                        FLAGS.crop_size,
                        FLAGS.rgb_channels
                        )

        with tf.variable_scope('Flow'):
            logit, _ = InceptionI3d(
                                num_classes=FLAGS.classics,
                                spatial_squeeze=True,
                                final_endpoint='Mixed_5c',
                                name='inception_i3d'
                                )(flow_images_placeholder, is_training)
        norm_score = tf.nn.softmax(logit)

        # Create a saver for writing training checkpoints.

        saver = tf.train.Saver()
        init = tf.global_variables_initializer()

        # Create a session for running Ops on the Graph.
        sess = tf.Session(
                        config=tf.ConfigProto(allow_soft_placement=True)
                        )
        sess.run(init)

    ckpt = tf.train.get_checkpoint_state(pre_model_save_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        print ("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
        print ("load complete!")

    all_steps = num_test_videos
    top1_list = []
    count = 0
    
    for step in xrange(all_steps):
        start_time = time.time()
        s_index = 0
        predicts = []
        data = {}
        data_new = np.array([])
        top1 = False
        while True:
            _, val_images, val_labels, s_index, is_end = input_test.read_clip_and_label(
                            filename=file[step],
                            batch_size=FLAGS.batch_size * gpu_num,
                            s_index=s_index,
                            num_frames_per_clip=FLAGS.num_frame_per_clib,
                            crop_size=FLAGS.crop_size,
                            )
            data_features = sess.run(norm_score,
                               feed_dict={
                                            flow_images_placeholder: val_images,
                                            labels_placeholder: val_labels,
                                            is_training: False
                                            })
            
            temp = np.mean(data_features, axis=2)
            tmp = np.mean(temp, axis=2)
            tmp = np.squeeze(tmp)
            

            if data_new.shape[0] == 0:
                data_new = tmp
            else:
                data_new = np.concatenate([data_new,tmp])

            labell = np.zeros((1,51))
            labell[0,int(val_labels)] = 1

            #print(labell)

            if is_end:
                ndata = int(data_new.shape[0])
                
                if ndata <=nn:
                    data_new = np.concatenate([data_new,data_new],  axis=0)
                    data_new = np.concatenate([data_new,data_new],  axis=0)
                    ndata = int(data_new.shape[0])
                
                sampling = ndata/nn
                sampling = int(sampling)
                ndata2 = data_new[0:nn*sampling,:]
                ndata3 = ndata2[0::sampling,:]
                data['feat'] = ndata3
                data['label'] = labell
                
                temp = file[step].split("/")
                namefile = temp[6].split(" ")
                
                datasave = outfile+namefile[0]
                np.save(datasave, data)
                break
        count = count+1
                    
        print(count,' >>' ,namefile[0])
        


def main(_):
    run_training()


if __name__ == '__main__':
    tf.app.run()


Number of test videos=1530
net_shape:
(1, 8, 7, 7, 1024)
./best_models/flow_imagenet_40000_6_64_0.0001_decay2/i3d_hmdb_model-39999
loading checkpoint ./best_models/flow_imagenet_40000_6_64_0.0001_decay2/i3d_hmdb_model-39999,waiting......
INFO:tensorflow:Restoring parameters from ./best_models/flow_imagenet_40000_6_64_0.0001_decay2/i3d_hmdb_model-39999
load complete!
1  >> TVs_Best_Kisses_Top_50_52_to_41_kiss_h_nm_np2_le_goo_1
2  >> Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_0
3  >> Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_2
4  >> Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_f_cm_np1_ri_med_4
5  >> Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_u_cm_np1_le_med_1
6  >> Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_u_cm_np1_le_med_3
7  >> Florian_Fromlowitz_beim_Training_der_U_21_Nationalmannschaft_catch_u_cm_np1_le_med_5
8  >> Aussie_Brunette_Br

124  >> Aishwarya_Rai_Burns_David_Letterman_in_his_own_show_clap_u_cm_np1_fr_bad_1
125  >> Alex_applauding_himself_clap_u_cm_np1_fr_med_1
126  >> Alex_applauding_himself_clap_u_nm_np1_fr_med_0
127  >> Finding_Forrester_3_clap_u_nm_np1_ri_med_0
128  >> LetThePeopleClap_clap_u_cm_np1_fr_med_1
129  >> LetThePeopleClap_clap_u_cm_np1_fr_med_2
130  >> Martin_klatscht_in_die_h_nde_und_FURZT_clap_u_cm_np1_fr_med_0
131  >> Return_of_the_King_12_clap_u_nm_np1_fr_goo_4
132  >> Standing_clap_f_cm_np1_fr_med_0
133  >> Standing_clap_f_cm_np1_fr_med_1
134  >> The_Slow_Clap_clap_u_cm_np1_fr_med_3
135  >> The_Slow_Clap_clap_u_cm_np1_fr_med_4
136  >> The_Slow_Clap_clap_u_cm_np2_fr_med_17
137  >> The_Slow_Clap_clap_u_nm_np1_fr_bad_20
138  >> The_Slow_Clap_clap_u_nm_np1_fr_bad_25
139  >> The_Slow_Clap_clap_u_nm_np1_fr_bad_27
140  >> The_Slow_Clap_clap_u_nm_np1_fr_med_0
141  >> The_Slow_Clap_clap_u_nm_np1_fr_med_1
142  >> The_Slow_Clap_clap_u_nm_np1_fr_med_12
143  >> The_Slow_Clap_clap_u_nm_np1_fr_med_14
1

246  >> Iaido_13_katas_draw_sword_f_nm_np1_fr_med_0
247  >> Iaido_13_katas_draw_sword_f_nm_np1_fr_med_12
248  >> Iaido_13_katas_draw_sword_f_nm_np1_fr_med_7
249  >> Iaido_13_katas_draw_sword_f_nm_np1_le_med_5
250  >> MAF_Tenshin_Ryu_draw_sword_f_cm_np1_fr_med_3
251  >> Muso_Jikiden_Eishinryu_in_Guldental_draw_sword_f_cm_np1_ba_med_2
252  >> Muso_Jikiden_Eishinryu_in_Guldental_draw_sword_f_cm_np1_le_med_3
253  >> Muso_Jikiden_Eishinryu_in_Guldental_draw_sword_f_cm_np1_le_med_4
254  >> Muso_Jikiden_Eishinryu_in_Guldental_draw_sword_f_cm_np1_ri_med_1
255  >> Nakayama_Aikido_Iaido_Part_3_draw_sword_f_cm_np1_le_med_0
256  >> Samurai_Fight_Scene_draw_sword_f_cm_np2_fr_goo_2
257  >> Tenshin_Ryu_Aikido_draw_sword_f_nm_np1_ri_med_0
258  >> The_Last_Samurai_-_In_Your_Honor_draw_sword_u_nm_np1_fr_med_6
259  >> The_Three_Musketeers_1993_Final_Battle_1_3_draw_sword_u_cm_np1_fr_med_1
260  >> The_Three_Musketeers_1993_Final_Battle_1_3_draw_sword_u_nm_np3_ri_med_0
261  >> Ukigumo_Kata_from_Hasegawa_Ei

389  >> The_Matrix_Revolutions_6_fall_floor_f_cm_np1_fr_bad_3
390  >> Veoh_Alpha_Dog_1_fall_floor_f_cm_np1_fr_med_35
391  >> Veoh_Alpha_Dog_1_fall_floor_u_cm_np1_fr_med_46
392  >> Advanced_Rapier_fencing_f_cm_np2_le_bad_0
393  >> Advanced_Rapier_fencing_f_cm_np2_le_bad_3
394  >> Advanced_Rapier_fencing_f_cm_np2_le_bad_5
395  >> Advanced_Rapier_fencing_f_cm_np2_ri_bad_1
396  >> Advanced_Rapier_fencing_f_cm_np2_ri_bad_2
397  >> Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_f_cm_np2_le_goo_0
398  >> Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_u_cm_np2_ba_goo_1
399  >> Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_u_cm_np2_fr_goo_2
400  >> Die_Another_Day_-_Fencing_Scene_Part_1_[HD]_fencing_u_cm_np2_fr_goo_3
401  >> Epee_Fencing_Attacks_-_Remises_fencing_f_nm_np2_le_bad_2
402  >> Hamlet_1996_Fencing_Scenes_fencing_f_cm_np2_fr_med_1
403  >> Hamlet_1996_Fencing_Scenes_fencing_u_cm_np2_fr_med_2
404  >> Hamlet_1996_Fencing_Scenes_fencing_u_cm_np2_ri_med_4
405  >> Rapier_at_

520  >> Destruction_of_a_TV_hit_f_cm_np1_ri_bad_2
521  >> Destruction_of_a_TV_hit_f_cm_np1_ri_bad_3
522  >> Drum_Beats_-_Drum_Beats-_Slow_Blues_hit_u_cm_np1_le_bad_2
523  >> HARD_FACE_SLAP_COMPILATION_-_THE_FUNNIEST_FACE_SLAPS_EVER_-_Old_school_Vids_hit_u_cm_np1_ba_bad_1
524  >> HARD_FACE_SLAP_COMPILATION_-_THE_FUNNIEST_FACE_SLAPS_EVER_-_Old_school_Vids_hit_u_cm_np1_ri_bad_2
525  >> Joey_Jordison_drum_solo_hit_u_cm_np1_ba_bad_0
526  >> Joey_Jordison_drum_solo_hit_u_cm_np1_ba_bad_2
527  >> Joey_Jordison_drum_solo_hit_u_cm_np1_le_bad_1
528  >> Schnaaper_Crew__Eisbaden_2007_hit_f_cm_np1_ba_bad_4
529  >> Schnaaper_Crew__Eisbaden_2007_hit_f_cm_np1_fr_bad_6
530  >> Schnaaper_Crew__Eisbaden_2007_hit_f_cm_np1_le_bad_0
531  >> Schnaaper_Crew__Eisbaden_2007_hit_f_cm_np1_le_bad_1
532  >> Schnaaper_Crew__Eisbaden_2007_hit_f_cm_np1_le_bad_2
533  >> SweeneyTodd_hit_u_nm_np1_fr_bad_2
534  >> TV_Recycling_hit_f_cm_np1_fr_bad_5
535  >> TV_Recycling_hit_f_cm_np1_le_bad_0
536  >> TV_Recycling_hit_f_cm_np

...

1187  >> SMILES_FROM_OFF_THE_ROAD_2_-TREASURING_THE_GIRLCHILD_smile_h_cm_np1_fr_bad_1
1188  >> SMILES_FROM_OFF_THE_ROAD_2_-TREASURING_THE_GIRLCHILD_smile_h_nm_np1_fr_goo_2
1189  >> SMILES_FROM_OFF_THE_ROAD_2_-TREASURING_THE_GIRLCHILD_smile_h_nm_np1_fr_med_0
1190  >> SMILES_FROM_OFF_THE_ROAD_2_-TREASURING_THE_GIRLCHILD_smile_h_nm_np1_fr_med_3
1191  >> SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_bad_2
1192  >> SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_goo_0
1193  >> SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_goo_3
1194  >> SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_med_1
1195  >> SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_med_4
1196  >> SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_med_5
1197  >> Two_Towers_3_smile_h_nm_np1_fr_goo_7
1198  >> showyoursmile_smile_h_nm_np1_fr_goo_0
1199  >> smile_collection_7_smile_h_nm_np1_fr_goo_0
1200  >> smile_collection_7_smi

1327  >> Highlander_I_with_Christophe_Lambert__Sean_Connery_sword_exercise_f_cm_np2_ba_med_1
1328  >> Kendo_Basics_sword_exercise_f_cm_np1_le_bad_2
1329  >> Kendo_Basics_sword_exercise_f_cm_np2_le_bad_3
1330  >> Kendo_Basics_sword_exercise_f_nm_np1_fr_bad_5
1331  >> Kendo_Basics_sword_exercise_f_nm_np1_le_bad_4
1332  >> Kendo_Basics_sword_exercise_f_nm_np1_ri_bad_6
1333  >> Kendo_Basics_sword_exercise_f_nm_np1_ri_bad_7
1334  >> Kendo_Basics_sword_exercise_u_cm_np1_fr_bad_1
1335  >> MAF_Tenshin_Ryu_sword_exercise_f_nm_np1_ri_bad_2
1336  >> MAF_Tenshin_Ryu_sword_exercise_f_nm_np2_fr_bad_4
1337  >> Spanish_counters_to_Italian_fencing_sword_exercise_f_cm_np2_le_bad_2
1338  >> Spanish_counters_to_Italian_fencing_sword_exercise_f_cm_np2_ri_bad_1
1339  >> VERGIL_DEVIL_MAY_CRY_swinging_his_sword_sword_exercise_f_cm_np1_fr_med_1
1340  >> VERGIL_DEVIL_MAY_CRY_swinging_his_sword_sword_exercise_f_cm_np1_fr_med_3
1341  >> VERGIL_DEVIL_MAY_CRY_swinging_his_sword_sword_exercise_f_cm_np1_fr_med_5
1342

1472  >> AmericanGangster_walk_f_nm_np1_ba_med_72
1473  >> AmericanGangster_walk_f_nm_np1_fr_med_38
1474  >> AmericanGangster_walk_f_nm_np1_fr_med_6
1475  >> AmericanGangster_walk_u_cm_np1_ba_med_37
1476  >> AmericanGangster_walk_u_nm_np1_fr_med_15
1477  >> BIG_FISH_walk_f_nm_np1_fr_med_2
1478  >> Eurotrip_walk_f_nm_np1_fr_med_8
1479  >> EverythingisIlluminated_walk_f_nm_np1_le_med_0
1480  >> HP_PRISONER_OF_AZKABAN_walk_f_cm_np1_ba_med_23
1481  >> HP_PRISONER_OF_AZKABAN_walk_f_cm_np1_ri_med_2
1482  >> Hitch_Part_1_walk_u_cm_np1_fr_med_0
1483  >> IamLegend_walk_f_nm_np1_fr_med_17
1484  >> LONGESTYARD_walk_f_nm_np1_fr_med_6
1485  >> MeettheFockers_walk_f_nm_np1_ba_med_9
1486  >> MeettheFockers_walk_u_cm_np1_fr_goo_7
1487  >> Panic_in_the_Streets_walk_u_cm_np1_ba_med_5
1488  >> Panic_in_the_Streets_walk_u_nm_np1_fr_med_4
1489  >> Prelinger_HabitPat1954_walk_f_nm_np6_fr_med_20
1490  >> Prelinger_LetYours1940_walk_f_nm_np1_ba_med_5
1491  >> RATRACE_walk_f_cm_np1_ba_med_17
1492  >> SafeInPor

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
